In [14]:
import wrds

In [39]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [13]:
def pull_Option_info(wrds_username = WRDS_USERNAME, year = 1996): 
	sql_query = f"""
		SELECT 
			a.date, a.secid, a.days,  
			a.cp_flag, a.strike_price, a.forward_price
		FROM
			optionm_all.stdopd{year} AS a
		WHERE 
			a.strike_price BETWEEN .8*a.forward_price and 1.2*a.forward_price
			AND
			a.days BETWEEN '0' and '100'
		LIMIT 1000
	""" 
	#LIMIT 1000
	db = wrds.Connection(wrds_username=wrds_username)
	optm = db.raw_sql(sql_query, date_cols = ["date"])
	db.close()
	return optm


NameError: name 'WRDS_USERNAME' is not defined

In [15]:
def pull_Option_info(wrds_username, year = 1996): 
	sql_query = f"""
		SELECT *
		FROM
			optionm_all.stdopd{year} AS a
		LIMIT 1000
	""" 

	db = wrds.Connection(wrds_username=wrds_username)
	optm = db.raw_sql(sql_query, date_cols = ["date"])
	db.close()
	return optm

In [16]:
conn = wrds.Connection(wrds_username="hholt")
conn.list_tables(library="optionm_all")

Loading library list...
Done


['distrd',
 'distrprojd1996',
 'distrprojd1997',
 'distrprojd1998',
 'distrprojd1999',
 'distrprojd2000',
 'distrprojd2001',
 'distrprojd2002',
 'distrprojd2003',
 'distrprojd2004',
 'distrprojd2005',
 'distrprojd2006',
 'distrprojd2007',
 'distrprojd2008',
 'distrprojd2009',
 'distrprojd2010',
 'distrprojd2011',
 'distrprojd2012',
 'distrprojd2013',
 'distrprojd2014',
 'distrprojd2015',
 'distrprojd2016',
 'distrprojd2017',
 'distrprojd2018',
 'distrprojd2019',
 'distrprojd2020',
 'distrprojd2021',
 'distrprojd2022',
 'distrprojd2023',
 'exchgd',
 'fwdprd1996',
 'fwdprd1997',
 'fwdprd1998',
 'fwdprd1999',
 'fwdprd2000',
 'fwdprd2001',
 'fwdprd2002',
 'fwdprd2003',
 'fwdprd2004',
 'fwdprd2005',
 'fwdprd2006',
 'fwdprd2007',
 'fwdprd2008',
 'fwdprd2009',
 'fwdprd2010',
 'fwdprd2011',
 'fwdprd2012',
 'fwdprd2013',
 'fwdprd2014',
 'fwdprd2015',
 'fwdprd2016',
 'fwdprd2017',
 'fwdprd2018',
 'fwdprd2019',
 'fwdprd2020',
 'fwdprd2021',
 'fwdprd2022',
 'fwdprd2023',
 'hvold1996',
 'hvold1997'

In [47]:
conn.get_table(library="optionm_all", table="opprcd1996",obs=5)

,secid,date,symbol,symbol_flag,exdate,last_date,cp_flag,strike_price,best_bid,best_offer,volume,open_interest,impl_volatility,delta,gamma,vega,theta,optionid,cfadj,am_settlement,contract_size,ss_flag,forward_price,expiry_indicator,root,suffix
0,5005.0,1996-01-04,09C04.71,0,1996-05-18,None,C,5000.0,3.500,3.8750,0.0,10.0,0.499353,0.977071,0.021003,0.282112,-0.450118,10224753.0,1.0,0.0,100.0,0,None,None,09C04,71
1,5005.0,1996-01-04,099AF.44,0,1996-08-17,None,P,7500.0,0.750,0.9375,0.0,10.0,0.565265,-0.282751,0.091322,2.264206,-0.893695,10071876.0,1.0,0.0,100.0,0,None,None,099AF,44
2,5005.0,1996-01-04,09B86.5D,0,1996-02-17,None,C,10000.0,0.250,0.3750,0.0,423.0,0.652115,0.290704,0.178798,1.006999,-2.910794,10192477.0,1.0,0.0,100.0,0,None,None,09B86,5D
3,5005.0,1996-01-04,09D85.65,0,1996-02-17,None,C,5000.0,3.375,3.7500,0.0,0.0,NaN,NaN,NaN,NaN,NaN,10323301.0,1.0,0.0,100.0,0,None,None,09D85,65
4,5005.0,1996-01-04,09FB8.95,0,1996-01-20,1996-01-04,C,10000.0,0.000,0.1250,40.0,200.0,0.618655,0.123897,0.190533,0.355180,-2.730374,10467477.0,1.0,0.0,100.0,0,None,None,09FB8,95


In [46]:
conn.get_table(library="optionm_all", table="stdopd1996",obs=5)

,secid,date,days,forward_price,strike_price,premium,impl_volatility,delta,gamma,theta,vega,cp_flag
0,5005.0,1996-01-04,182.0,8.798315,8.798315,1.401746,0.574742,-0.430845,0.117781,-1.148857,2.359826,P
1,5005.0,1996-01-04,182.0,8.798315,8.798315,1.525858,0.637896,0.589062,0.100806,-1.695364,2.352038,C
2,5005.0,1996-01-04,152.0,8.760547,8.760547,1.296590,0.582622,-0.435151,0.126633,-1.299982,2.163659,P
3,5005.0,1996-01-04,152.0,8.760547,8.760547,1.474057,0.673929,0.586038,0.104588,-1.935919,2.153158,C
4,5005.0,1996-01-04,122.0,8.722377,8.722377,1.161659,0.583340,-0.441320,0.140742,-1.482216,1.944851,P


In [3]:
df = pull_Option_info("hholt")

Loading library list...
Done


In [26]:
opinfd.loc[opinfd['secid']==108105]

,secid,div_convention,exercise_style,am_set_flag
4014,108105.0,I,E,None


In [4]:
df.head()

,secid,date,days,forward_price,strike_price,premium,impl_volatility,delta,gamma,theta,vega,cp_flag
0,5005.0,1996-01-04,182.0,8.798315,8.798315,1.401746,0.574742,-0.430845,0.117781,-1.148857,2.359826,P
1,5005.0,1996-01-04,182.0,8.798315,8.798315,1.525858,0.637896,0.589062,0.100806,-1.695364,2.352038,C
2,5005.0,1996-01-04,152.0,8.760547,8.760547,1.296590,0.582622,-0.435151,0.126633,-1.299982,2.163659,P
3,5005.0,1996-01-04,152.0,8.760547,8.760547,1.474057,0.673929,0.586038,0.104588,-1.935919,2.153158,C
4,5005.0,1996-01-04,122.0,8.722377,8.722377,1.161659,0.583340,-0.441320,0.140742,-1.482216,1.944851,P


In [11]:
dup_cols = ['cp_flag','strike_price']

duplicates = df[df.duplicated(subset=dup_cols, keep=False)]


In [12]:
duplicates.head()

,secid,date,days,forward_price,strike_price,premium,impl_volatility,delta,gamma,theta,vega,cp_flag
12,5005.0,1996-01-04,10.0,8.575951,0.0,0.0,NaN,NaN,NaN,NaN,NaN,P
13,5005.0,1996-01-04,10.0,8.575951,0.0,0.0,NaN,NaN,NaN,NaN,NaN,C
26,5005.0,1996-01-05,10.0,8.513353,0.0,0.0,NaN,NaN,NaN,NaN,NaN,P
27,5005.0,1996-01-05,10.0,8.513353,0.0,0.0,NaN,NaN,NaN,NaN,NaN,C
28,5005.0,1996-01-08,10.0,8.638476,0.0,0.0,NaN,NaN,NaN,NaN,NaN,C
